In [5]:
import sys
import time
import tracemalloc


class Tep:
    """Là một đối tượng gồm các thông tin của item tại gian dịch (TID, xác xuất, tiện ích, tiện ích giao dịch)
    """
    def __init__(self, TID, prob, util, trans_util):
        """khởi tạo tep

        Args:
            TID (int): mã giao dịch
            prob (float): xác xuất tồn tại
            util (int): giá trị tiện ích
            trans_util (int): tiện ích giao dịch
        """
        self.TID = TID
        self.prob = round(prob, 2)
        self.util = util
        self.trans_util = trans_util

    def combine_with(self, other_tep):
        """kết hợp 2 tep với nhau

        Args:
            other_tep (Tep): Tep cần kết hợp

        Returns:
            Tep: một tep mới
        """
        combined_prob = round(self.prob * other_tep.prob, 2)
        combined_util = self.util + other_tep.util
        return Tep(self.TID, combined_prob, combined_util, other_tep.trans_util)

    def __repr__(self):
        return f"Tep(TID={self.TID}, prob={self.prob}, util={self.util}, trans_util={self.trans_util})\n"


class Cup:
    """Cup là đối tượng chứa các thông tin của một item bao gồm: tên, độ hỗ trợ mong đợi, danh sách Tep,
        giá trị xác xuất lớn nhất trong Tep, TWU, giá trị tiện ích, phần tử cuối của item
    """
    def __init__(self, name, tep_list=None):
        """khởi tạo Cup

        Args:
            name (str): tên của cup (tức tên của item)
            tep_list (List[Tep], None): danh sách Tep của item
        """
        self.name = name
        self.exp_sup = round(sum(tep.prob for tep in tep_list), 2) if tep_list else 0
        self.tep_list = tep_list if tep_list else []
        self.max_prob = round(max(tep.prob for tep in tep_list), 2) if tep_list else 0
        self.trans_wei_util = sum(tep.trans_util for tep in tep_list) if tep_list else 0
        self.utility = sum(tep.util for tep in tep_list) if tep_list else 0
        self.last = []

    def update(self, probability, TID, trans_util, util_value):
        """Cập nhật lại các giá trị trong Cup

        Args:
            probability (float): xác xuất tồn tại
            TID (_type_): mã giao dịch
            trans_util (_type_): tiện ích giao dịch
            util_value (_type_): giá trị tiện ích
        """
        probability = round(probability, 2)  # Round to 2 decimal places
        tep = Tep(TID, probability, util_value, trans_util)
        self.exp_sup = round(self.exp_sup + probability, 2)  # Round after addition
        self.utility += util_value
        self.tep_list.append(tep)
        self.max_prob = max(self.max_prob, probability)
        self.trans_wei_util += trans_util

    def combine_tep(self, tep_list_x, tep_list_y):
        """Kết hợp hai danh sách Tep lại với tạo thành danh sách Tep mới

        Args:
            tep_list_x (List[Tep]): danh sách Tep của item X
            tep_list_y (List[Tep]): danh sách Tep của item Y

        Returns:
            List[Tep]: danh sách Tep của item XY
        """
        tep_list_xy = []
        i, j = 0, 0
        while i < len(tep_list_x) and j < len(tep_list_y):
            tX = tep_list_x[i]
            tY = tep_list_y[j]
            if tX.TID < tY.TID:
                i += 1
            elif tX.TID > tY.TID:
                j += 1
            else:
                combined_prob = round(tX.prob * tY.prob, 2)
                combined_util = tX.util + tY.util
                tep_list_xy.append(Tep(tX.TID, combined_prob, combined_util, tX.trans_util))
                i += 1
                j += 1
        return tep_list_xy

    def combine_with(self, other_cup):
        """Kết hợp hai Cup lại với nhau

        Args:
            other_cup (Cup): Cup tham gia kết hợp

        Returns:
            Cup: một Cup mới
        """
        if len(other_cup.last) == 0:
            combined_name = self.name + ', ' + other_cup.name
            combined_tep_list = self.combine_tep(self.tep_list, other_cup.tep_list)
            last = [other_cup]
        else:
            combined_name = self.name + ', ' + other_cup.last[0].name
            combined_tep_list = self.combine_tep(self.tep_list, other_cup.last[0].tep_list)
            last = other_cup.last

        combined_cup = Cup(combined_name, combined_tep_list)
        combined_cup.last = last
        return combined_cup

    def __repr__(self):
        return f"Cup(name={self.name}, exp_sup={self.exp_sup}, utility={self.utility})"



class AlgorithmTUHUFP:
    """Lớp này triển khai thuật toán khai thác top-k mẫu phổ biến tiện ích cao không chắc chắn
    """
    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.database_util = 0
        self.candidates = 0
        self.top_UHUFP = []
        self.single_cup = {}
        self.threshold = float('-inf')
        self.min_util = float('-inf')
        self.peak_memory_usage = 0

    def read_data(self, file_path, percentage, k):
        """đọc các giá trị từ cơ sở dũ liệu

        Args:
            file_path (str): đường dẫn chứa file cơ sở dữ liệu
            percentage (float): ngưỡng tiện ích (%)
            k (int): số lượng mẫu cần tìm
        """
        file_paths = file_path.split(", ")
        # print(f"File paths: {file_paths}")  # Check the paths
        try:
            with open(file_paths[0], 'r') as file1, open(file_paths[1], 'r') as file2:
                print("Reading data . . .")
                tlines = file1.readlines()
                ulines = file2.readlines()
                item_name = tlines[0].strip().split(" ")
                # print(f"Item names: {item_name}")  # Check item names

                TID = 1
                for prob_line, util_line in zip(tlines[1:], ulines):
                    # print(f"Processing TID {TID}: {prob_line.strip()} | {util_line.strip()}")  # Check each line
                    self.process_data(item_name, prob_line.strip(), util_line.strip(), TID)
                    TID += 1
                    self.database_size += 1

                self.min_util = int(self.database_util * percentage)
                print(f"Minimum utility set to: {self.min_util}")
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            print("STOP ALGORITHM !!!")
            sys.exit(0)
        except Exception as e:
            print(f"An error occurred: {e}")
            sys.exit(0)

            
    def process_data(self, item_name, prob_line, util_line, TID):
        """Xử lí dữ liệu và tạo Cup

        Args:
            item_name (List[str]): danh sách các item
            prob_line (float): danh sách các xác xuất của item
            util_line (int): danh sách tiện ích của item
            TID (int): mã giao dịch
        """
        prob_list = prob_line.split(" ")
        trans = util_line.split(":")
        items_util = trans[0].split(" ")
        # print(items_util)
        trans_util = int(trans[1])
        util_list = trans[2].split(" ")

        item_util_list = {item: int(util) for item, util in zip(items_util, util_list)}


        self.database_util += trans_util

        for i, prob in enumerate(prob_list):
            prob = prob.strip()
            if prob and float(prob) > 0:
                try:
                    item = item_name[i]
                    probability = round(float(prob), 2)  # Round to 2 decimal places
                   
                    util_value = item_util_list[item]
                    cup_name = item
                    #kiểm tra cup đã được tạo hay chưa
                    if cup_name in self.single_cup:
                        self.single_cup[cup_name].update(probability, TID, trans_util, util_value)
                    else:
                        new_tep = Tep(TID, probability, util_value, trans_util)
                        self.single_cup[cup_name] = Cup(cup_name, [new_tep])
                except ValueError as ve:
                    print(f"Error processing probability '{prob}': {ve}")
                    continue



    def combine_cup(self, cupX, cupY):
        """Kết hơp hai cup X và Y

        Args:
            cupX (Cup): Cup tham gia kết hợp
            cupY (Cup): Cup tham gia kết hợp

        Returns:
           Cup: CupXY
        """
        combined_cup = cupX.combine_with(cupY)
        combined_cup.exp_sup = round(combined_cup.exp_sup, 2)  # Round after combining
        return combined_cup
    
    def get_first_UHUFP(self, min_util, k):
        """Lấy danh sách top k đầu tiên và danh sách ứng viên tham gia

        Args:
            min_util (int): ngưỡng tiện ích
            k (int): số lượng mẫu cần tìm

        Returns:
            List[Cup]: danh sách các ứng viên
        """
        candidate_list = []
        cups = sorted(self.single_cup.values(), key=lambda x: x.exp_sup, reverse=True)
        # chỉ lấy k Cup đầu tiên
        for cup in cups[:k]:
            if cup.trans_wei_util >= min_util:
                candidate_list.append(cup)
            if cup.utility >= min_util:
                self.top_UHUFP.append({'name': cup.name, 'exp_sup': cup.exp_sup, 'utility': cup.utility})
        return candidate_list



    def TUHUFPSearchHelper(self, combined, k):
        """Kiểm tra mẫu có phải là mẫu tiện ích cao hay không và thêm vào top k

        Args:
            combined (Cup): mẫu cần kiểm tra
            k (int): số lượng mẫu cần tìm
        """
        if combined.utility >= self.min_util:
            self.top_UHUFP.append({'name': combined.name, 'exp_sup': combined.exp_sup, 'utility': combined.utility})
            self.top_UHUFP.sort(key=lambda x: x['exp_sup'], reverse=True)
            if len(self.top_UHUFP) > k:
                self.top_UHUFP.pop()
                self.threshold = self.top_UHUFP[-1]['exp_sup']
            if len(self.top_UHUFP) == k:
                self.threshold = self.top_UHUFP[-1]['exp_sup']
        # print(f"Top {k} CUPs after add {combined}")
        # for cup in self.top_UHUFP[:]:
        #     print(cup)
        # print(self.threshold)


    def TUHUFPSearch(self, currentCup, k):
        """Phương thức chính của thuật toán, khai thác các mẫu UHUFP

        Args:
            currentCup (List[Cup]): danh sách ứng viên tham gia
            k (int): số lượng mẫu cần tìm
        """
        if len(currentCup) <= 1:
            # dừng thuật toán khi không còn ứng viên tham gia
            return
        # duyệt từng ứng viên
        for i in range(len(currentCup) - 1):
            newCupList = []
            for j in range(i + 1, len(currentCup)):
                # chiến lược 3
                overestimate = currentCup[i].exp_sup * currentCup[j].max_prob
                if overestimate < self.threshold:
                    break
                combined = self.combine_cup(currentCup[i], currentCup[j])
                # kiểm tra và thêm vào top k nếu thỏa điều kiện
                if combined.exp_sup > self.threshold:
                    self.TUHUFPSearchHelper(combined, k)
                    # thêm các ứng viên mới
                    if combined.trans_wei_util >= self.min_util:
                        newCupList.append(combined)
                        self.candidates += 1
            # tìm kiếm với danh sách ứng viên mới
            self.TUHUFPSearch(newCupList, k)


    def run_TUHUFP_algorithm(self, file_path, percentage, k):
        """Chạy thuật toán

        Args:
            file_path (str): đường dẫn tới file cơ sở dữ liệu
            percentage (_type_): ngưỡng tiện ích (%)
            k (int): số lượng mẫu tham gia
        """
        self.start_timestamp = time.time()
        self.candidates = 0
        self.database_size = 0

        self.read_data(file_path, percentage, k)
        print({'database util: ': self.database_util})

        # danh sách ứng viên
        candidate_list = self.get_first_UHUFP(self.min_util, k)
        if not self.single_cup:
            print("No CUP List was created. STOP ALGORITHM !!!")
            sys.exit(0)
        self.candidates = len(candidate_list)
        # print(self.candidates)

        # đặt threhold
        if len(self.top_UHUFP) == k:
            self.threshold = self.top_UHUFP[-1]['exp_sup']

        # print(f"Top {k} CUPs after sorting by exp_sup:")
        # i = 1
        # for cup in candidate_list[:]:
        #     print(f" {i}: {cup}")
        #     i +=1

        # Bắt đầu theo dõi bộ nhớ
        tracemalloc.start()
        # bắt đầu tìm kiếm
        self.TUHUFPSearch(candidate_list, k)
        print(f"Top {k} CUPs:")
        i = 1
        for cup in self.top_UHUFP[:]:
            print(f" {i}: {cup}")
            i +=1
        # Kết thúc theo dõi bộ nhớ và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6  # Chuyển đổi sang MB

        self.end_timestamp = time.time()

    def print_stats(self, path):
        """Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """        
        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.min_util}\n")
            self.top_UHUFP.sort(key=lambda x: x['exp_sup'], reverse=True)
            for t in self.top_UHUFP:
                output_file.write(f"{t['name']}: {t['exp_sup']}: {t['utility']}\n")
            output_file.write("=============  TOP-K UFPs v1.20 - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidates}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  # Ghi lại peak memory usage






In [6]:
# chạy với dữ liệu mẫu
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("../../data/example.txt, ../../data/example_utility.txt", 0.3, 6)
algorithm.print_stats("../../out/TUHUFP/output_example.txt")

Reading data . . .
Minimum utility set to: 30
{'database util: ': 102}
6
Top 6 CUPs:
 1: {'name': 'd', 'exp_sup': 3.3, 'utility': 30}
 2: {'name': 'c, d', 'exp_sup': 1.95, 'utility': 38}
 3: {'name': 'd, a', 'exp_sup': 1.2, 'utility': 35}
 4: {'name': 'c, d, a', 'exp_sup': 0.98, 'utility': 42}
 5: {'name': 'c, d, b', 'exp_sup': 0.88, 'utility': 40}
 6: {'name': 'd, a, b', 'exp_sup': 0.55, 'utility': 33}


In [7]:
# chạy với foodmart
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("../../data/input_foodmart.txt, ../../data/foodmart_utility.txt", 0.0004, 300)
algorithm.print_stats("../../out/TUHUFP/output_foodmart_top_300.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
300
Top 300 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17

In [8]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("../../data/input_retail.txt, ../../data/retail_utility.txt", 0.0001, 300)
algorithm.print_stats("../../out/TUHUFP/output_retail_top_300.txt")

Reading data . . .


KeyboardInterrupt: 

In [ ]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("../../data/input_chess.txt, ../../data/chess_utility.txt", 0.001, 300)
algorithm.print_stats("../../out/TUHUFP/output_chess_top_300.txt")